<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>

### pandas 라이브러리와 탐색적 데이터 분석 과정 익히기

> 다양한 데이터 분석 케이스를 통해 데이터 분석과 pandas 라이브러리 활용에 대해 익히보기로 합니다.

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">탐색적 데이터 분석</font><br>

### 전체 판매 프로세스
1. 해당 쇼핑몰에 중소업체가 계약을 맺고
2. 중소업체가 해당 쇼핑몰에 직접 상품을 올리고
2. 고객이 구매하면, 중소업체가 Olist가 제공하는 물류 파트너를 활용해서 배송을 하고,
3. 고객이 상품을 받으면, 고객에게 이메일 survey 가 전송되고,
4. 고객이 이메일 survey 에 별점과 커멘트를 남겨서 제출하게 됨
    
### 데이터 출처
- 브라질에서 가장 큰 백화점의 이커머스 쇼핑몰 (https://olist.com/)
  - 2016년도부터 2018년도 9월까지의 100k 개의 구매 데이터 정보
  - 구매 상태, 가격, 지불수단, 물류 관련, 리뷰관련, 상품 정보, 구매자 지역 관련 정보

### 주요 질문(탐색하고자 하는 질문 리스트)
- 얼마나 많은 고객이 있는가?
- 고객은 어디에 주로 사는가?
- 고객은 주로 어떤 지불방법을 사용하는가?
- **평균 거래액은 얼마일까?**
- 일별, 주별, 월별 판매 트렌드는?
- 어떤 카테고리가 가장 많은 상품이 팔렸을까?
</div>

In [ ]:
import pandas as pd
PATH = "00_data/"

In [ ]:
products = pd.read_csv(PATH + "olist_products_dataset.csv", encoding='utf-8-sig')
customers = pd.read_csv(PATH + "olist_customers_dataset.csv", encoding='utf-8-sig')
geolocation = pd.read_csv(PATH + "olist_geolocation_dataset.csv", encoding='utf-8-sig')
order_items = pd.read_csv(PATH + "olist_order_items_dataset.csv", encoding='utf-8-sig')
payments = pd.read_csv(PATH + "olist_order_payments_dataset.csv", encoding='utf-8-sig')
reviews = pd.read_csv(PATH + "olist_order_reviews_dataset.csv", encoding='utf-8-sig')
orders = pd.read_csv(PATH + "olist_orders_dataset.csv", encoding='utf-8-sig')
sellers = pd.read_csv(PATH + "olist_sellers_dataset.csv", encoding='utf-8-sig')
category_name = pd.read_csv(PATH + "product_category_name_translation.csv", encoding='utf-8-sig')

### 1. 평균 거래액은 얼마일까?

> 월별 평균 거래액 분석

### 1.1. 데이터 분석 전 해야할 일
- Data cleansing or data cleaning
- 가장 기본적인 것은 없는 데이터를 확인하는 일

In [ ]:
orders.head()

In [ ]:
order_items.head()

In [ ]:
payments.head()

> orders 의 구매 날짜와 payments 의 총 구매 금액을 가지고 월별 평균 거래액 분석을 하기로 함

In [ ]:
orders.info()

#### 없는 데이터 확인하기
- isnull().sum() 사용

In [ ]:
orders.isnull().sum()

#### 모든 데이터가 있는 데이터만 공식적인 데이터로 사용하기로 함
- 없는 데이터 삭제하기

In [ ]:
orders = orders.dropna()

In [ ]:
orders.isnull().sum()

In [ ]:
orders.info()

- payments 는 없는 데이터가 없는 상태

In [ ]:
payments.isnull().sum()

#### orders 와 payments 사이즈 비교

In [ ]:
payments.info()

In [ ]:
orders.info()

#### order_id 중 중복된 데이터가 있는지 확인
- value_counts(): 각 값이 전체에서 중복된 횟수를 리턴 (unique할 경우, 1을 리턴)
- max(): 최대값 가져오기
- value_counts().max(): 최대 중복된 데이터의 횟수 리턴

In [ ]:
orders['order_id'].value_counts().max()

In [ ]:
payments['order_id'].value_counts().max()

In [ ]:
payments['order_id'].value_counts()

In [ ]:
payments[payments['order_id'] == 'fa65dad1b0e818e3ccc5cb0e39231352']

#### 중복된 order_id 에 대한 지불 가격을 합치기로 함
> 중복된 order_id 에 대해 orders 필드값이 덮어씌워져서 본래 orders 보다 많은 행이 생긴 것임

In [ ]:
payments = payments.groupby('order_id').sum()
payments[payments.index == 'fa65dad1b0e818e3ccc5cb0e39231352']

#### orders 의 구매 날짜와 payments 의 총 지불 금액을 합침

In [ ]:
merged_order = pd.merge(orders, payments, on='order_id')

In [ ]:
merged_order.info()

In [ ]:
merged_order[merged_order['order_id'] == 'fa65dad1b0e818e3ccc5cb0e39231352']

### 1.2 pandas 로 날짜 다루기

#### 시계열 자료와 pandas
- 년도별, 월별, 일별, 시별, 분별 초별등 시간의 흐름에 따라 관측된 자료
- pandas 에서 시계열 자료를 손쉽게 다룰 수 있도록, datetime (datetime64) 자료형 제공
  - pandas.to_datetime() 함수를 사용해서, 날짜와 시간을 나타내는 문자열을 datetime (datetime64) 자료형으로 변경 가능

#### pandas.to_datetime() 사용법
- 문자열 타입의 시간을 pandas 의 datetime (datetime64) 형으로 변경 
- 주요 사용법
  - Series 변수 = to_datetime(Series 변수)
    - return 된 Series 변수 데이터는 datetime64 형으로 변형되어 저장
  - Series 변수 = to_datetime(Series 변수, format='~~~')
    - Series 에 변환될 문자열이 특별한 포맷을 가져서, 자동변환이 어려운 경우 명시적으로 format 지정 (옵션)
  - Series 변수 = to_datetime(Series 변수, errors='raise')
    - 디폴트 raise
    - errors 가능한 값: ignore(무시), raise(에러 발생), coerce(NaT 로 값 변경해서 저장) (옵션)


| 표시 | 의미                                |
|------|:-------------------------------------|
| %y   | 연도를 축약해서 표시, 예: 21        |
| %Y   | 연도를 축약하지 않고 표시, 예: 2021 |
| %m   | 월을 두자릿 수로 표시, 예: 01 ~ 12  |
| %d   | 일을 두자릿 수로 표시, 예: 01 ~ 31  |
| %H   | 시간 표시(24시간), 예: 00 ~ 23        |
| %M   | 분 표시, 예: 00 ~ 59  |
| %S   | 초 표시, 예: 00 ~ 59        |

> 참고: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
import pandas as pd
df = pd.DataFrame({'order':['2020-01-01 07:10:00',
                            '2020-01-08 07:20:30',
                            '2020-02-20 11:20:00',
                            '2020-02-20 04:40:50',
                            '2020-02-28 12:10:20',
                            '2019-01-10 17:23:50',
                            '2019-06-20 22:27:50',
                            '2019-06-20 21:15:59',
                            '2019-12-10 21:15:59',
                           ]})

In [ ]:
df.info()

In [ ]:
df['order'] = pd.to_datetime(df['order'], format='%Y-%m-%d %H:%M:%S', errors='raise') 
#df['order'] = pd.to_datetime(df['order'], format='%Y-%m-%d %H:%M:%S', errors='raise')

In [ ]:
df.info()

### order 한 시간 정보 데이터만 datetime64 로 변환하기

In [ ]:
# 지금까지 작성한 부분을 한데 모아서 한번에 실행 (주피터 노트북 중간부터 들으신다면...)
import pandas as pd
PATH = "00_data/"
payments = pd.read_csv(PATH + "olist_order_payments_dataset.csv", encoding='utf-8-sig')
orders = pd.read_csv(PATH + "olist_orders_dataset.csv", encoding='utf-8-sig')
orders = orders.dropna()
payments = payments.groupby('order_id').sum()
merged_order = pd.merge(orders, payments, on='order_id')

In [ ]:
merged_order.info()

In [ ]:
merged_order.head(1)

In [ ]:
merged_order['order_purchase_timestamp'] = pd.to_datetime(merged_order['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise') 

In [ ]:
merged_order.info()

### pandas.DataFrame.copy
- 데이터프레임 중 일부를 선택 후, 조작하면 해당 데이터프레임도 변경
- copy() 를 통해, 복사본을 만들어서 조작하여, 원본 데이터프레임은 보존 가능

In [ ]:
merged_order_payment_date = merged_order[['order_purchase_timestamp', 'payment_value']].copy()

In [ ]:
merged_order_payment_date.head()

### pandas.Grouper
- pandas groupby 명령에 보다 세부적인 grouping 이 가능토록 하는 명령
- pandas groupby 함수와 함께 쓰여서, 시간별로 데이터를 분류할 수 있는 기능
- 특정 시간별로 grouping 할 수 있음

```
데이터프레임.groupby(pd.Groper(key='그루핑기준이되는 컬럼', freq='세부 기준'))

```

### 1.3. 시간대별 거래액 확인하기

### 1.3.1. 월별 거래액 확인하기

In [ ]:
merged_order_month_sum = merged_order_payment_date.groupby(pd.Grouper(key='order_purchase_timestamp', freq='ME')).sum() # key 는 기본이 index 임
merged_order_month_sum.head()

### 시각화해서 트렌드 확인하기

In [ ]:
import plotly.express as px

# merged_order_month_sum: 인덱스에 날짜, 'payment_value' 열에 결제 금액 합계가 있음
fig = px.bar(
    x=merged_order_month_sum.index,        # x축: 주문 구매 날짜 (인덱스)
    y=merged_order_month_sum['payment_value'],  # y축: 결제 금액 합계
    title="Monthly Payment Value Sum",     # 그래프 제목
    labels={
        'x': 'Order Purchase Timestamp',   # x축 레이블
        'y': 'Payment Value'                 # y축 레이블
    }
)

# 긴 날짜 레이블이 겹치지 않도록 x축 라벨 각도 조정 (선택 사항)
fig.update_layout(xaxis_tickangle=-45)

# 그래프 출력
fig.show()


<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
    <font size="3em" style="font-weight:bold;color:#3f8dbf;">월별 평균 거래액</font>
</div>

In [ ]:
merged_order_month_sum['payment_value'].mean()

In [ ]:
merged_order_month_sum.head()

In [ ]:
merged_order_month_sum['payment_value'][3:].mean()

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
    <font size="3em" style="font-weight:bold;color:#3f8dbf;">최대 거래액을 기록한 월</font>
</div>

In [ ]:
merged_order_month_sum[merged_order_month_sum['payment_value'] == merged_order_month_sum['payment_value'].max()]

### 1.4. 월별 거래액 시각화 
- 현업 수준의 디테일이 살아 있는 그래프 그려보기

### 지금까지의 데이터 전처리

In [ ]:
# 지금까지 작성한 부분을 한데 모아서 한번에 실행 (주피터 노트북 중간부터 들으신다면...)
import pandas as pd
PATH = "00_data/"
payments = pd.read_csv(PATH + "olist_order_payments_dataset.csv", encoding='utf-8-sig')
orders = pd.read_csv(PATH + "olist_orders_dataset.csv", encoding='utf-8-sig')
orders = orders.dropna()
payments = payments.groupby('order_id').sum()
merged_order = pd.merge(orders, payments, on='order_id')
merged_order['order_purchase_timestamp'] = pd.to_datetime(merged_order['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise') 
merged_order_payment_date = merged_order[['order_purchase_timestamp', 'payment_value']].copy()
merged_order_month_sum = merged_order_payment_date.groupby(pd.Grouper(key='order_purchase_timestamp', freq='ME')).sum() # key 는 기본이 index 임

#### 1. plotly 로 세부적인 부분까지 수정해보기

In [ ]:
merged_order_month_sum

- 각 필드 확인: https://plotly.com/python/reference/

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=merged_order_month_sum.index, 
        y=merged_order_month_sum['payment_value'], 
        text=merged_order_month_sum['payment_value'], 
        textposition='auto', 
        texttemplate='R$ %{text:.0f}'
    )
)

fig.update_layout(
    {
        "title": {
            "text": "<b>Turnover per Month in Brazilian Olist E-Commerce company</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "from Oct. 2016 to Sep. 2018",
            "showticklabels":True,
            "dtick": "M1",
            "tickfont": {
                "size": 7                
            }
        },
        "yaxis": {
            "title": "Turnover per Month"
        }
    }
)

fig.show()

#### 2. 불필요한 데이터 삭제

In [ ]:
merged_order_month_sum_from2017 = merged_order_month_sum[merged_order_month_sum.index > '2017-01-01']
merged_order_month_sum_from2017

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=merged_order_month_sum_from2017.index, 
        y=merged_order_month_sum_from2017['payment_value'], 
        text=merged_order_month_sum_from2017['payment_value'], 
        textposition='auto', 
        texttemplate='R$ %{text:.0f}'
    )
)

fig.update_layout(
    {
        "title": {
            "text": "<b>Turnover per Month in Brazilian Olist E-Commerce company</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "from Jan. 2017 to Sep. 2018",
            "showticklabels":True,
            "dtick": "M1",
            "tickfont": {
                "size": 7                
            }
        },
        "yaxis": {
            "title": "Turnover per Month"
        }
    }
)

fig.show()

#### 3. 그래프 테마 변경해보기

In [ ]:
import plotly.io as pio
pio.templates

In [ ]:
import plotly.graph_objects as go
for template in pio.templates:
    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            x=merged_order_month_sum_from2017.index, 
            y=merged_order_month_sum_from2017['payment_value'], 
            text=merged_order_month_sum_from2017['payment_value'], 
            textposition='auto', 
            texttemplate='R$ %{text:.0f}'
        )
    )
    fig.update_layout(
        {
            "title": {
                "text": "<b>Turnover per Month in Brazilian Olist E-Commerce company</b> by " + template,
                "x": 0.5,
                "y": 0.9,
                "font": {
                    "size": 15
                }
            },
            "xaxis": {
                "title": "from Feb. 2017 to Sep. 2018",
                "showticklabels":True,
                "tick0": "2017-01-31", # 처음 tick 을 설정을 해주지 않을 경우, x 축이 밀리는 경우가 있음
                "dtick": "M1", # 한 달 단위로 tick 설정
                "tickfont": {
                    "size": 7                
                }
            },
            "yaxis": {
                "title": "Turnover per Month"
            },
            "template":template
        }
    )
    fig.show()

#### 4. 원하는 테마로 최종 선택하기

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=merged_order_month_sum_from2017.index, 
        y=merged_order_month_sum_from2017['payment_value'], 
        text=merged_order_month_sum_from2017['payment_value'], 
        textposition='auto', 
        texttemplate='R$ %{text:,.0f}'
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>Turnover per Month in Brazilian Olist E-Commerce company</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "from Jan. 2017 to Sep. 2018",
            "showticklabels":True,
            "tick0": "2017-01-31", # 처음 tick 을 설정을 해주지 않을 경우, x 축이 밀리는 경우가 있음
            "dtick": "M1", # 한 달 단위로 tick 설정
            "tickfont": {
                "size": 7                
            }
        },
        "yaxis": {
            "title": "Turnover per Month"
        },
        "template":'plotly_white'
    }
)
fig.show()

#### 5. bar 색상 바꾸기 (최대 거래액을 가진 달은 별도 색상으로 변경하기)

- 참고 사이트: https://color.adobe.com/ko/trends

In [ ]:
colors = ['#03588C',] * len(merged_order_month_sum_from2017.index)
colors[10] = '#F24472'

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=merged_order_month_sum_from2017.index, 
        y=merged_order_month_sum_from2017['payment_value'], 
        text=merged_order_month_sum_from2017['payment_value'], 
        textposition='auto', 
        texttemplate='R$ %{text:,.0f}',
        marker_color=colors
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>Turnover per Month in Brazilian Olist E-Commerce company</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "from Jan. 2017 to Aug. 2018",
            "showticklabels":True,
            "tick0": "2017-01-31",
            "dtick": "M1",
            "tickfont": {
                "size": 7                
            }
        },
        "yaxis": {
            "title": "Turnover per Month",
            "tickfont": {
                "size": 10                
            }
        },
        "template":'plotly_white'
    }
)

fig.show()

#### 6. annotation 추가하기

- 참고 사이트: https://plotly.com/python/text-and-annotations/
- 상세 옵션: https://plotly.com/python/reference/#layout-annotations

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=merged_order_month_sum_from2017.index, 
        y=merged_order_month_sum_from2017['payment_value'], 
        text=merged_order_month_sum_from2017['payment_value'], 
        textposition='auto', 
        texttemplate='R$ %{y:,.0f}',
        marker_color=colors
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>Turnover per Month in Brazilian Olist E-Commerce company</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "from Jan. 2017 to Sep. 2018",
            "showticklabels":True,
            "tick0": "2017-01-31",
            "dtick": "M1",
            "tickfont": {
                "size": 7                
            }
        },
        "yaxis": {
            "title": "Turnover per Month",
            "tickfont": {
                "size": 10                
            }
        },
        "template":'plotly_white'
    }
)

fig.add_annotation(
            x="2017-11-30",
            y=1153393,
            text="<b>Peaked Monthly Turnover</b>",
            showarrow=True,
            font=dict(
                size=10,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#77CFD9",
            ax=20,
            ay=-30,
            bordercolor="#77CFD9",
            borderwidth=2,
            borderpad=4,
            bgcolor="#F25D50",
            opacity=0.9
)

fig.show()

<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>